# Making dataframes for VADs to keep track of everything and ensure that I am only keeping VADs

This is a neater conglomoration of notebooks (plot_attempts, identify_vads). It will make dataframes and radar objects that one can use in other notebooks. THIS IS STEP 1 AFTER DEALIASING THE FILES AND USING TOBAC

In [318]:
import pyart
import numpy as np
from datetime import datetime, timedelta
import glob
# from radarcalc import *
import matplotlib.pyplot as plt
import pandas as pd
import metpy.calc as mpcalc
import metpy
import metpy.plots
from metpy.units import units
import cartopy.crs as ccrs
import gc
from astropy.convolution import convolve
from boto.s3.connection import S3Connection
import tempfile
import copy
import matplotlib
import xarray as xr
import math
from datetime import datetime

In [323]:
def dealias_Ka(radar,PPIGC_flag=False):
    '''
    This function aims to take care of all the nitty gritty customizations of pyarts dealiasing specifically for TTUKa radar data.

    Parameter: radar (pyart object), PPIGC_flag (boolean *kwarg)
    Returns: radar (pyart object) with corrected_velocity field and velocity_texture field added on

    How it works:
    1. Calculates velocity texture and creates a histogram based on all of the magnitudes of textures at each bin for the entire volume
    2. loops through each indivual sweep
    3. infinite loop to iterate between the minimum amount of texture between textures 1 and 6 and the maximum amount of texture in the histogram, with 0.5 m/s steps
    4. Each iteration, create a gatefilter that filters the bins above that texture value and dealias it with the gatefilter to ignore the high textured regions
    5. If the maximum texture is reached before breaking, set the gatefilter to mask textures above 12 m/s
    6. if the scan is an RHI, run a 2 pass variance filter along each ray.
        This convolves a 71 sized boxcar with the data, and if the difference between the point the boxcar is centered on and the mean of the boxcar is greater than the nyquist, then add/subtract 2*nyquist to that point
    7. Then, take the difference between the mean of the bottom 4 rays of the dealiased velocity in the RHI and the bottom 4 rays of the aliased velocity
    8. If the absolute difference is larger than the nyquist, the either subtract 2* nyquist or add 2*nyquist to the entire sweep and break the loop
    9. If the absolute difference is less than the nyquist, no fixes need to be applied, break the loop
    10. If the scan is a PPI, do steps 8 and 9, but skip steps 6 and 7
    11. Outside the infinite loop, add the option of the PPIGC_flag, a boolean flag that will help maintain ground clutter in PPIs at 0 m/s
        This works by identifying regions of very low spectrum width (<0.1 m/s), and setting the velocity in those regions = 0. Please note, this may introduce artificial speckles of 0 in real data, where spectrum width is noisy
    12. Apply the alias fix algorithm which convolves a boxcar of specified boxcar of size 9 with the data, and if the variance between the middle pixel of interest and the mean is greater than the nyquist, then flip it back over
    12. At the end of each sweep, assign the data from that processed sweep into a dictionary, then add the dictionary as the corrected_velocity field.
    
    The aforementioned method is FAR from perfect, but is as robust as I can do currently. One thing to improve this though is to use the UNRAVEL algorithm: https://github.com/vlouf/dealias
    The UNRAVEL algorithm shows remarkable error characteristics compared to "competitors", possibly at a time cost, which isn't a HUGE deal for us. Downside is it may not work for our "volumes" since they are temporally uncorrelated and are not full volumes through the atmosphere
    '''
    
    vel_texture = pyart.retrieve.calculate_velocity_texture(radar, vel_field='velocity', wind_size=3)
    radar.add_field('velocity_texture', vel_texture, replace_existing=True)
    hist, bins = np.histogram(radar.fields['velocity_texture']['data'][~np.isnan(radar.fields['velocity_texture']['data'])], bins=150)
    bins = (bins[1:]+bins[:-1])/2.0
    gatefilter = pyart.filters.GateFilter(radar)
    velocity_dealiased = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=radar.instrument_parameters['nyquist_velocity']['data'][0], centered=True) #standin, data will be replaced

    for swp_id in range(radar.nsweeps):
        #get indices from beginning and ending of sweep
        sw_start = radar.sweep_start_ray_index['data'][swp_id]
        sw_end = radar.sweep_end_ray_index['data'][swp_id]+1

        counter = 0
        while True: #do an infinite loop and either break it when the data is unfolded correctly or when the max texture is reached
            #if the bin with the lowest count between textures 1 and 6 + i*0.5 is less than the maximum amount of bins
            if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                gatefilter.exclude_above('velocity_texture', bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5)
                nyq = radar.instrument_parameters['nyquist_velocity']['data'][0]
                vede = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=nyq,
                                                                        centered=True, gatefilter=gatefilter)
            else:
                gatefilter.exclude_above('velocity_texture', 12)#bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]])
                nyq = radar.instrument_parameters['nyquist_velocity']['data'][0]
                vede = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=nyq,
                                                                        centered=True, gatefilter=gatefilter)


            np.ma.set_fill_value(vede['data'], np.nan)
            #extract mask so we can apply the correct gatefilters on later
            mask=np.ma.getmask(vede['data'])

            #apply mask to velocity field and fix the small blips from dealiasing
            if radar.scan_type == 'rhi':
                #pass 1 of variance filtering along the ray.
                #Convolves a 71 sized boxcar with the data, and if the difference between the point the boxcar is centered on and the mean of the boxcar is greater than the nyquist, then add/subtract 2*nyquist to that point
                vel = vede['data'].filled()

                for sw in range(np.shape(vel)[0]):
                    mean = convolve(vel[sw,:],np.ones(71))
                    var = vel[sw,:]-mean
                    high_idx = var > nyq
                    low_idx = var < -nyq
                    vel[sw,:][high_idx] = vel[sw,:][high_idx] - 2*nyq
                    vel[sw,:][low_idx] = vel[sw,:][low_idx] + 2*nyq
                vede['data']=np.ma.masked_array(vel,mask=mask,fill_value=np.nan)

                #pass 2 of variance filtering along the ray. In case there are errant folds than need to be folded back
                vel = vede['data'].filled()
                for sw in range(np.shape(vel)[0]):
                    mean = convolve(vel[sw,:],np.ones(71))
                    var = vel[sw,:]-mean
                    high_idx = var > nyq
                    low_idx = var < -nyq
                    vel[sw,:][high_idx] = vel[sw,:][high_idx] - 2*nyq
                    vel[sw,:][low_idx] = vel[sw,:][low_idx] + 2*nyq
                vede['data']=np.ma.masked_array(vel,mask=mask,fill_value=np.nan)

                #find means of the bottom 4 rays of the RHI(should be close to 0) and compare the dealiased velocities to the aliased velocities
                np.ma.set_fill_value(radar.fields['velocity']['data'], np.nan)
                meanvelal = np.mean(radar.fields['velocity']['data'][sw_start:sw_start+4,:].filled()[~np.isnan(radar.fields['velocity']['data'][sw_start:sw_start+4,:].filled())])
                meanveldeal = np.mean(vede['data'][sw_start:sw_start+4,:].filled()[~np.isnan(vede['data'][sw_start:sw_start+4,:].filled())])
                if np.abs(meanvelal-meanveldeal) < nyq: #nyq is an arbitrary threshold and should be tuned
                    break
                if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                    if (meanvelal-meanveldeal) > 0:
                        vede['data'][sw_start:sw_end,:] += 2*nyq
                    else:
                        vede['data'][sw_start:sw_end,:] -= 2*nyq
                    break
            if radar.scan_type == 'ppi':
                np.ma.set_fill_value(radar.fields['velocity']['data'], np.nan)
                meanvelal = np.mean(radar.fields['velocity']['data'][sw_start:sw_end,:].filled()[~np.isnan(radar.fields['velocity']['data'][sw_start:sw_end,:].filled())])
                meanveldeal = np.mean(vede['data'][sw_start:sw_end,:].filled()[~np.isnan(vede['data'][sw_start:sw_end,:].filled())])
                if np.abs(meanvelal-meanveldeal) < nyq: #nyq is an arbitrary threshold and should be tuned
                    break
                if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                    if (meanvelal-meanveldeal) > 0:
                        vede['data'][sw_start:sw_end,:] += 2*nyq
                    else:
                        vede['data'][sw_start:sw_end,:] -= 2*nyq
                    break
            counter+=1
            
        #put alias fix inside here instead of calling it to make it more portable
        delta=3
        mean = convolve(vede['data'][sw_start:sw_end,:],np.ones((delta,delta))/delta**2.)
        mean[0,:] = vede['data'][sw_start:sw_end,:][0,:]
        mean[-1,:] = vede['data'][sw_start:sw_end,:][-1,:]
        var = vede['data'][sw_start:sw_end,:] - mean

        high_idx = np.logical_and(var > nyq, var < 4*nyq)
        low_idx = np.logical_and(var < -nyq, var > -4*nyq)

        vede['data'][sw_start:sw_end,:][high_idx] = vede['data'][sw_start:sw_end,:][high_idx] - 2*nyq
        vede['data'][sw_start:sw_end,:][low_idx] = vede['data'][sw_start:sw_end,:][low_idx] + 2*nyq

        #corrects ground clutter by arbitrarily setting the velocity equal to 0 where spectrum width is less than 0.075 m/s
        if PPIGC_flag == True:
            if radar.scan_type == 'ppi':
                sw = radar.fields['spectrum_width']['data'][sw_start:sw_end,:].filled()
                vel = radar.fields['velocity']['data'][sw_start:sw_end,:].filled()
                mask = sw<0.1
                vede['data'][sw_start:sw_end,:] = np.where(~mask,vede['data'][sw_start:sw_end,:],0)

        velocity_dealiased['data'][sw_start:sw_end,:] = vede['data'][sw_start:sw_end,:]
        velocity_dealiased['data'][sw_start:sw_end,:] = alias_fix(velocity_dealiased['data'][sw_start:sw_end,:],nyq,delta=9)
    radar.add_field('corrected_velocity', velocity_dealiased, replace_existing=True)

    return radar

def alias_fix(vel,nyq,delta=3):
    '''
    !!!!!!!!!!!!!!!!!!
    Removes dealiasing errors around the periphery of a folded region

    Parameters: velocity array (array), nyquist velocity (number), size of window (int, must be odd, unity is no change)
    Returns: cleaned velocity array (array)
    '''
    mean = convolve(vel,np.ones((delta,delta))/delta**2.)
    mean[0,:] = vel[0,:]
    mean[-1,:] = vel[-1,:]
    var = vel - mean

    high_idx = np.logical_and(var > nyq, var < 4*nyq)
    low_idx = np.logical_and(var < -nyq, var > -4*nyq)

    vel[high_idx] = vel[high_idx] - 2*nyq
    vel[low_idx] = vel[low_idx] + 2*nyq

    return vel

def get_radar_from_aws(site, datetime_t, datetime_te):
    """
    Get the closest volume of NEXRAD data to a particular datetime.
    Parameters
    ----------
    site : string
        four letter radar designation
    datetime_t : datetime
        desired date time
    Returns
    -------
    radar : Py-ART Radar Object
        Radar closest to the queried datetime
    """

    # First create the query string for the bucket knowing
    # how NOAA and AWS store the data
    my_pref = datetime_t.strftime('%Y/%m/%d/') + site

    # Connect to the bucket
    conn = S3Connection(anon = True)
    bucket = conn.get_bucket('noaa-nexrad-level2')

    # Get a list of files
    bucket_list = list(bucket.list(prefix = my_pref))

    # we are going to create a list of keys and datetimes to allow easy searching
    keys = []
    datetimes = []

    # populate the list
    for i in range(len(bucket_list)):
        this_str = str(bucket_list[i].key)
        if 'gz' in this_str:
            endme = this_str[-22:-4]
            fmt = '%Y%m%d_%H%M%S_V0'
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])

        if this_str[-3::] == 'V06':
            endme = this_str[-19::]
            fmt = '%Y%m%d_%H%M%S_V06'
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])

    # find the closest available radar to your datetime
    closest_datetime_b = _nearestDate(datetimes, datetime_t)
    closest_datetime_e = _nearestDate(datetimes, datetime_te)

    index_b = datetimes.index(closest_datetime_b)
    index_e = datetimes.index(closest_datetime_e)

    radar_namelist = keys[index_b:index_e+1]
    radar_list=[]
    for i in range(np.shape(radar_namelist)[0]):
        localfile = tempfile.NamedTemporaryFile()
        radar_namelist[i].get_contents_to_filename(localfile.name)
        radar_list.append(pyart.io.read(localfile.name))
    return radar_namelist,radar_list

def getLocation(lat1, lon1, brng, distancekm):
    lat1 = lat1 * np.pi / 180.0
    lon1 = lon1 * np.pi / 180.0
    #earth radius
    R = 6378.1
    #R = ~ 3959 MilesR = 3959
    bearing = (brng / 90.)* np.pi / 2.

    lat2 = np.arcsin(np.sin(lat1) * np.cos(distancekm/R) + np.cos(lat1) * np.sin(distancekm/R) * np.cos(bearing))
    lon2 = lon1 + np.arctan2(np.sin(bearing)*np.sin(distancekm/R)*np.cos(lat1),np.cos(distancekm/R)-np.sin(lat1)*np.sin(lat2))
    lon2 = 180.0 * lon2 / np.pi
    lat2 = 180.0 * lat2 / np.pi
    return lat2, lon2

def _nearestDate(dates, pivot):
    return min(dates, key=lambda x: abs(x - pivot))

def find_nearest(array, value):
    '''
    Function to find index of the array in which the value is closest to

    Parameters: array (array), value (number)
    Returns: index (int)

    Example: xind = CM1calc.find_nearest(x,5)
    '''

    array = np.asarray(array)
    idx = (np.abs(array-value)).argmin()
    return idx


def vehicle_correction_vad(radar,df):
    '''
    Function that creates a 'vad_corrected_velocity' field that can be used for vad calculations, 
    but should be general enough to use for stationary VADs as well as moving PPIs. 
    Other than adding the new field, the radar times are smoothly interpolated and the azimuths are 
    corrected via the GPS pandas dataframe.
    
    Parameters: pyart radar object (object), pandas dataframe of appropriate radarGPS file (dataframe)
    Returns: pyart radar object (object), speed (float), speed variance (float), bearing (float), bearing variance (float), 
             latitude (float), latitude variance (float), longitude (float), longitude variance (float)
    
    Example: radar, velmean, velvar, bearmean, bearvar, latmean, latvar, lonmean, lonvar = vehicle_correction_vad(radar,df)

    p.s. only works if the velocity is already dealiased and there is a 'corrected_velocity' field
         also only works if a single sweep is extracted, example: radar = radar.extract_sweeps([0])
    '''
    #for i in df:
    #orders the time to increase monotonically instead of having a massive step jump in the middle
    roll_mag = (np.argmax(np.abs(np.gradient(radar.time['data'])))+1)
    times = np.roll(radar.time['data'],-roll_mag) 
    
    #a complicated way to create linear increasing times (instead of steps) that start at 0 seconds after the time datum and increase to the middle of the second max time plateau (if confused, plotting it is helpful)
    #from now on, we are going to assume ray_times is the fractional seconds after the time datum the ray is gathered, and we need to roll it back to match with the rest of the data
    ray_times = np.roll(np.arange(0,((np.unique(times)[-2])/(find_nearest(times,np.unique(times)[-2])+int(np.sum(radar.time['data']==np.unique(times)[-2])/2)))*len(times)+1e-11,((np.unique(times)[-2])/(find_nearest(times,np.unique(times)[-2])+int(np.sum(radar.time['data']==np.unique(times)[-2])/2)))),roll_mag)

    radar.time['data']=ray_times
    
    #df['datetime'] = pd.to_datetime(df['ddmmyy']+df['hhmmss[UTC]'], format='%d%m%y%H%M%S')
    df['datetime'] = [datetime.strptime(d,'%d%m%y%H%M%S') for d in df['ddmmyy']+[f'{h:06}' for h in df['hhmmss[UTC]'].astype(int)]]
    beginscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index
    endscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.ceil(np.amax(ray_times))+1
    if len(beginscanindex) == 0: # MAKES IT SO THAT IF THE DATETIME IS MISSING IT SKIPS THE FILE
        return None#, None, None, None, None, None, None, None
    dfscan = df.iloc[beginscanindex[0].astype(int):endscanindex[0].astype(int)]
    dfscan = dfscan.astype({'Bearing[degrees]': 'float'})
    dfscan = dfscan.astype({'Velocity[knots]': 'float'})
        
    ray_bearings = np.interp(ray_times,np.arange(len(dfscan)),dfscan['Bearing[degrees]'])
    ray_speeds = np.interp(ray_times,np.arange(len(dfscan)),dfscan['Velocity[knots]'])
        
    #    print('velocity [kts]',dfscan['Velocity[knots]'].mean(),'+-',dfscan['Velocity[knots]'].var())
    speed = dfscan['Velocity[knots]'].mean()
    #    print('bearing',dfscan['Bearing[degrees]'].mean(),'+-',dfscan['Bearing[degrees]'].var())
    bearing = dfscan['Bearing[degrees]'].mean()
    #    print('latitude',dfscan['Latitude'].astype(float).mean(),'+-',dfscan['Latitude'].astype(float).var())
    lat = dfscan['Latitude'].astype(float).mean()
    #    print('longitude',dfscan['Longitude'].astype(float).mean(),'+-',dfscan['Longitude'].astype(float).var())
    lon = dfscan['Longitude'].astype(float).mean()
        
    radar.azimuth['data'] += ray_bearings[:-1] #bearing
        
    rad_vel = copy.deepcopy(radar.fields['corrected_velocity'])
        
    rad_vel['data']+=(np.cos(np.deg2rad(radar.azimuth['data']-ray_bearings[:-1]))*(ray_speeds[:-1]/1.94384)*np.cos(np.deg2rad(radar.fixed_angle['data'][0])))[:,np.newaxis]
        
    #fix mask, remove points very close to radar as well as the very last bin, more often than not, = bad data
    rad_vel['data'].mask[:,:5] = True
    rad_vel['data'].mask[:,-1] = True
    radar.add_field('vad_corrected_velocity', rad_vel, replace_existing=True)
        
    return radar, dfscan['Velocity[knots]'].mean(),dfscan['Velocity[knots]'].var(),dfscan['Bearing[degrees]'].mean(),dfscan['Bearing[degrees]'].var(),dfscan['Latitude'].astype(float).mean(),dfscan['Latitude'].astype(float).var(),dfscan['Longitude'].astype(float).mean(),dfscan['Longitude'].astype(float).var()
                

In [320]:
# all_dealiased_data_ka1 = sorted(glob.glob('/Users/juliabman/Desktop/dealiased_data/05232022/ka1/*.nc'))
# all_dealiased_data_ka2 = sorted(glob.glob('/Users/juliabman/Desktop/dealiased_data/05232022/ka2/*.nc'))
# the above paths are different, they may contain vads that arent stationary
all_dealiased_data_ka1 = sorted(glob.glob('/Users/juliabman/Desktop/dealiased_data/ka1/*.nc'))
all_dealiased_data_ka2 = sorted(glob.glob('/Users/juliabman/Desktop/dealiased_data/test2/*.nc'))
ka1gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka1_20220523.txt')
ka2gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka2_20220523.txt')
tobac_file = '/Users/juliabman/Desktop/research2024/tobac_Save/Track.nc'

d=20220523
pwd = '/Users/juliabman/Desktop/research2024'
df1 = pd.read_csv(f'{pwd}/{d}_Ka1_scan_log.txt')
df2 = pd.read_csv(f'{pwd}/{d}_Ka2_scan_log.txt')

In [4]:
np.size(all_dealiased_data_ka1)

307

In [5]:
np.size(all_dealiased_data_ka2)

317

In [6]:
ka1gps['ddmmyy'] = ka1gps['ddmmyy'].astype(str)
ka1gps['hhmmss[UTC]'] = ka1gps['hhmmss[UTC]'].astype(str)

ka2gps['ddmmyy'] = ka2gps['ddmmyy'].astype(str)
ka2gps['hhmmss[UTC]'] = ka2gps['hhmmss[UTC]'].astype(str)

In [7]:
df1.Speed

0      40.700357
1      41.957143
2      42.215926
3      27.442857
4       9.963333
         ...    
302     0.001688
303     0.001429
304     0.007662
305     0.003636
306     0.002436
Name: Speed, Length: 307, dtype: float64

In [8]:
corrected_vehicle_vads_ka1 = []
corrected_vads_file_names_ka1 = []
radar_array_ka1 = []
velmean_array_ka1 = []
velvar_array_ka1 = []
bearmean_array_ka1 = []
latmean_array_ka1 = []
latvar_array_ka1 = []
lonmean_array_ka1 = []
lonvar_array_ka1 = []
#time_of_scan_ka1 = []

for uncorrected_vad in all_dealiased_data_ka1:
    print(uncorrected_vad)
    read = pyart.io.read_cfradial(uncorrected_vad)
    
    radar, velmean, velvar, bearmean, bearvar, latmean, latvar, lonmean, lonvar = vehicle_correction_vad(read, ka1gps)
    #corrected_vehicle_vads_0523.append(correction_function)
    radar_array_ka1.append(radar)
    velmean_array_ka1.append(velmean)
    velvar_array_ka1.append(velvar)
    bearmean_array_ka1.append(bearmean)
    latmean_array_ka1.append(latmean)
    latvar_array_ka1.append(latvar)
    lonmean_array_ka1.append(lonmean)
    lonvar_array_ka1.append(lonvar)
    corrected_vads_file_names_ka1.append(uncorrected_vad)

/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523201819_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523201849_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523201919_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523201948_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523202019_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523205632_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523205701_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523205730_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523205759_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523205828_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523205857_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523205926_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523205955_dealiased.nc
/Users/juliabman/Desktop/dealiased_data/ka1/Ka12205

In [9]:
np.shape(corrected_vads_file_names_ka1)

(307,)

In [324]:
corrected_vehicle_vads_ka2 = []
corrected_vads_file_names_ka2 = []
radar_array_ka2 = []
velmean_array_ka2 = []
velvar_array_ka2 = []
bearmean_array_ka2 = []
latmean_array_ka2 = []
latvar_array_ka2 = []
lonmean_array_ka2 = []
lonvar_array_ka2 = []
#time_of_scan_ka2 = []

for uncorrected_vad_ka2 in all_dealiased_data_ka2:
    print(uncorrected_vad_ka2)
    read = pyart.io.read_cfradial(uncorrected_vad_ka2)
    all_vehicle_correction = vehicle_correction_vad(read, ka2gps)
    if all_vehicle_correction == None:
        print(uncorrected_vad_ka2.time)
        print(all_vehicle_correction)
        continue
    #corrected_vehicle_vads_0523.append(correction_function)
    radar_array_ka2.append(all_vehicle_correction[0])
    velmean_array_ka2.append(all_vehicle_correction[1])
    velvar_array_ka2.append(all_vehicle_correction[2])
    bearmean_array_ka2.append(all_vehicle_correction[3])
    latmean_array_ka2.append(all_vehicle_correction[5])
    latvar_array_ka2.append(all_vehicle_correction[6])
    lonmean_array_ka2.append(all_vehicle_correction[7])
    lonvar_array_ka2.append(all_vehicle_correction[8])
    corrected_vads_file_names_ka2.append(uncorrected_vad_ka2)

/Users/juliabman/Desktop/dealiased_data/ka2/Ka2220523205016_dealiased.nc


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U6')) -> None

In [11]:
np.size(corrected_vads_file_names_ka2)

258

In [12]:
dealiased_vad_ka1_times = []
for time_grab in range(len(corrected_vads_file_names_ka1)):
    file = corrected_vads_file_names_ka1[time_grab]
    time_yoink = file[47:-13]
    #print(time_yoink)
    time_yoink_datetime = datetime.strptime(time_yoink, '%y%m%d%H%M%S')
    dealiased_vad_ka1_times.append(time_yoink_datetime)

In [13]:
dealiased_vad_ka2_times = []
for time_grab2 in range(len(corrected_vads_file_names_ka2)):
    file2 = corrected_vads_file_names_ka2[time_grab2]
    time_yoink2 = file2[47:-13]
    #print(time_yoink2)
    time_yoink_datetime2 = datetime.strptime(time_yoink2, '%y%m%d%H%M%S')
    dealiased_vad_ka2_times.append(time_yoink_datetime2)

In [14]:
tobac_features_xr = xr.open_dataset(tobac_file)
idx = tobac_features_xr['idx'].data
cell = tobac_features_xr['cell'].data
morton_storm_indeces_idx = np.where(idx == 29)
morton_storm_indeces = np.where(cell == 29)
#print(morton_storm_indeces)
tobac_times = tobac_features_xr['time']
tobac_lats = np.array(tobac_features_xr['latitude'])
tobac_lons = np.array(tobac_features_xr['longitude'])

#morton_two_prior = morton_storm_indeces[405]
#morton_two_post = morton_storm_indeces[839]


morton_tobac_lats = tobac_lats[morton_storm_indeces]
morton_tobac_lons = tobac_lons[morton_storm_indeces]
morton_tobac_times = tobac_times[morton_storm_indeces]
morton_cell_idx_29 = cell[morton_storm_indeces]

#idx_29_lats = tobac_lats[morton_storm_indeces_idx]
#idx_29_lons = tobac_lons[morton_storm_indeces_idx]

morton_tobac_times_datetime = morton_tobac_times.astype('datetime64[s]')
#print(morton_tobac_times_datetime)

#print(morton_tobac_lats)
#print(morton_tobac_lons)

/var/folders/4f/9s_4__q16tx5t8vmww4mv03h0000gn/T/ipykernel_2574/451719395.py:23: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  morton_tobac_times_datetime = morton_tobac_times.astype('datetime64[s]')


In [15]:
# make arrays so we can do math with them
array_dealiased_vad_ka1_times = np.array(dealiased_vad_ka1_times)
array_tobac_times = np.array(tobac_times)

# make arrays into datetime
array_dealiased_vad_ka1_times_datetime = array_dealiased_vad_ka1_times.astype('datetime64[s]')
array_tobac_times_datetime = array_tobac_times.astype('datetime64[s]')

In [16]:
# make arrays so we can do math with them
array_dealiased_vad_ka2_times = np.array(dealiased_vad_ka2_times)
# make arrays into datetime
array_dealiased_vad_ka2_times_datetime = array_dealiased_vad_ka2_times.astype('datetime64[s]')

Now that we have all of our necessary info on our scans, we can combine them with the scan logs to select only the vads

In [17]:
d=20220523
pwd = '/Users/juliabman/Desktop/research2024'
df1 = pd.read_csv(f'{pwd}/{d}_Ka1_scan_log.txt')
df2 = pd.read_csv(f'{pwd}/{d}_Ka2_scan_log.txt')

In [141]:
df1

,Datetime,Scantype,Angles,Ranges,Duration,Latitude,Longitude,Heading,Speed,Distance,Deployment
0,2022-05-23 20:18:19,vad,[37. 45.],9972.0,27.0,33.593816,-102.339719,270.842857,40.700357,0.000000,0
1,2022-05-23 20:18:49,vad,[37. 45.],9972.0,27.0,33.593902,-102.346550,270.853214,41.957143,633.466135,0
2,2022-05-23 20:19:19,vad,[37. 45.],9972.0,26.0,33.593976,-102.353427,270.864074,42.215926,637.776940,0
3,2022-05-23 20:19:48,vad,[37. 45.],9972.0,27.0,33.594052,-102.359624,277.545714,27.442857,574.632380,0
4,2022-05-23 20:20:19,vad,[37. 45.],9972.0,26.0,33.594721,-102.361347,179.523333,9.963333,176.252456,0
...,...,...,...,...,...,...,...,...,...,...,...
302,2022-05-24 03:08:50,rhi,[263.2 273.2 283.2 293.2 303.2 313.2 323.2 333...,19965.0,76.0,33.592530,-102.035702,93.200000,0.001688,0.326354,10
303,2022-05-24 03:10:11,rhi,[263.2 273.2 283.2 293.2 303.2 313.2 323.2 333...,19965.0,76.0,33.592530,-102.035702,93.200000,0.001429,0.460026,10
304,2022-05-24 03:11:32,rhi,[263.2 273.2 283.2 293.2 303.2 313.2 323.2 333...,19965.0,76.0,33.592530,-102.035702,93.200000,0.007662,0.882721,10
305,2022-05-24 03:12:53,rhi,[263.2 273.2 283.2 293.2 303.2 313.2 323.2 333...,19965.0,76.0,33.592530,-102.035702,93.200000,0.003636,0.313107,10


In [18]:
scantype = df1.Scantype
datetime = df1.Datetime
angles = df1.Angles
ranges = df1.Ranges

In [19]:
datetime = datetime.astype('datetime64[s]')

In [139]:
datetime

0     2022-05-23 20:18:19
1     2022-05-23 20:18:49
2     2022-05-23 20:19:19
3     2022-05-23 20:19:48
4     2022-05-23 20:20:19
              ...        
302   2022-05-24 03:08:50
303   2022-05-24 03:10:11
304   2022-05-24 03:11:32
305   2022-05-24 03:12:53
306   2022-05-24 03:14:14
Name: Datetime, Length: 307, dtype: datetime64[s]

In [20]:
radar_column_ka1 = pd.Series(radar_array_ka1)
velmean_column_ka1 = pd.Series(velmean_array_ka1)
velvar_column_ka1 = pd.Series(velvar_array_ka1)
bearmean_column_ka1 = pd.Series(bearmean_array_ka1)
latmean_column_ka1 = pd.Series(latmean_array_ka1)
latvar_column_ka1 = pd.Series(latvar_array_ka1)
lonmean_column_ka1 = pd.Series(lonmean_array_ka1)
lonvar_column_ka1 = pd.Series(lonvar_array_ka1)
datetime_column_ka1 = pd.Series(array_dealiased_vad_ka1_times_datetime)

In [140]:
datetime_column_ka1

0     2022-05-23 20:18:19
1     2022-05-23 20:18:49
2     2022-05-23 20:19:19
3     2022-05-23 20:19:48
4     2022-05-23 20:20:19
              ...        
302   2022-05-24 03:08:50
303   2022-05-24 03:10:11
304   2022-05-24 03:11:32
305   2022-05-24 03:12:53
306   2022-05-24 03:14:14
Length: 307, dtype: datetime64[s]

In [21]:
corrected_scans_df_ka1 = pd.DataFrame(pd.concat([datetime, datetime_column_ka1, scantype, angles, ranges, radar_column_ka1, velmean_column_ka1, 
                                                 velvar_column_ka1, bearmean_column_ka1, latmean_column_ka1, latvar_column_ka1, 
                                                 lonmean_column_ka1, lonvar_column_ka1], axis=1))

In [22]:
corrected_scans_df_ka1.rename(columns={0: 'Datetime_VAD', 1: 'Radar', 2: 'Velmean', 3: 'Velvar', 4: 'Bearmean', 
                                        5: 'Latmean', 6: 'Latvar', 7: 'Lonmean', 8: 'Lonvar'}, inplace  = True)

In [136]:
corrected_scans_df_ka1

,Datetime,Datetime_VAD,Scantype,Angles,Ranges,Radar,Velmean,Velvar,Bearmean,Latmean,Latvar,Lonmean,Lonvar
0,2022-05-23 20:18:19,2022-05-23 20:18:19,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30000b6a0>,41.322128,13.541626,270.857660,33.593844,1.493618e-09,-102.341872,9.543316e-06
1,2022-05-23 20:18:49,2022-05-23 20:18:49,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30013a8b0>,42.164255,1.027729,270.870426,33.593926,1.237346e-09,-102.348762,1.018093e-05
2,2022-05-23 20:19:19,2022-05-23 20:19:19,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30005eb80>,39.817556,15.065437,270.855778,33.594000,1.142237e-09,-102.355451,8.773930e-06
3,2022-05-23 20:19:48,2022-05-23 20:19:48,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x105b14880>,2.022109,46.544900,298.949818,33.594931,5.005760e-08,-102.361993,4.267158e-07
4,2022-05-23 20:20:19,2022-05-23 20:20:19,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30073d070>,6.971087,18.566854,232.127826,33.594825,3.518888e-08,-102.361671,1.986989e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,2022-05-24 03:08:50,2022-05-24 03:08:50,rhi,[263.2 273.2 283.2 293.2 303.2 313.2 323.2 333...,19965.0,<pyart.core.radar.Radar object at 0x300ae1790>,0.002184,0.000108,77.670000,33.592543,2.042966e-28,-102.035679,4.665865e-12
303,2022-05-24 03:10:11,2022-05-24 03:10:11,rhi,[263.2 273.2 283.2 293.2 303.2 313.2 323.2 333...,19965.0,<pyart.core.radar.Radar object at 0x300e904c0>,0.004792,0.000373,77.670000,33.592543,5.101854e-29,-102.035684,2.476206e-12
304,2022-05-24 03:11:32,2022-05-24 03:11:32,rhi,[263.2 273.2 283.2 293.2 303.2 313.2 323.2 333...,19965.0,<pyart.core.radar.Radar object at 0x306b02d30>,0.006782,0.000457,77.670000,33.592543,2.042966e-28,-102.035694,1.302887e-11
305,2022-05-24 03:12:53,2022-05-24 03:12:53,rhi,[263.2 273.2 283.2 293.2 303.2 313.2 323.2 333...,19965.0,<pyart.core.radar.Radar object at 0x302087400>,0.003542,0.000173,77.670000,33.592543,5.101854e-29,-102.035696,9.473684e-13


In [166]:
corrected_scans_df_ka1 = corrected_scans_df_ka1.dropna()

In [305]:
corrected_vads_only_df_ka1_no_nan = corrected_scans_df_ka1[(corrected_scans_df_ka1.Scantype == 'vad')]

In [307]:
corrected_vads_only_df_ka1_no_nan

,Datetime,Datetime_VAD,Scantype,Angles,Ranges,Radar,Velmean,Velvar,Bearmean,Latmean,Latvar,Lonmean,Lonvar
0,2022-05-23 20:18:19,2022-05-23 20:18:19,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30000b6a0>,41.322128,13.541626,270.857660,33.593844,1.493618e-09,-102.341872,9.543316e-06
1,2022-05-23 20:18:49,2022-05-23 20:18:49,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30013a8b0>,42.164255,1.027729,270.870426,33.593926,1.237346e-09,-102.348762,1.018093e-05
2,2022-05-23 20:19:19,2022-05-23 20:19:19,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30005eb80>,39.817556,15.065437,270.855778,33.594000,1.142237e-09,-102.355451,8.773930e-06
3,2022-05-23 20:19:48,2022-05-23 20:19:48,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x105b14880>,2.022109,46.544900,298.949818,33.594931,5.005760e-08,-102.361993,4.267158e-07
4,2022-05-23 20:20:19,2022-05-23 20:20:19,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30073d070>,6.971087,18.566854,232.127826,33.594825,3.518888e-08,-102.361671,1.986989e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2022-05-24 00:01:10,2022-05-24 00:01:10,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x307afa580>,3.976230,137.229424,248.844976,33.793174,1.025092e-05,-102.644966,1.008538e-06
191,2022-05-24 00:01:39,2022-05-24 00:01:39,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x307d398b0>,43.196667,15.037367,38.183750,33.784412,8.276087e-06,-102.642297,2.191911e-09
192,2022-05-24 00:02:09,2022-05-24 00:02:09,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x307c26ac0>,30.318413,305.878072,94.438095,33.791226,7.861599e-06,-102.642176,2.957845e-09
193,2022-05-24 00:02:39,2022-05-24 00:02:39,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x307c267f0>,15.259574,295.846113,167.150213,33.793709,3.987920e-07,-102.642206,1.600216e-08


In [90]:
df2

,Datetime,Scantype,Angles,Ranges,Duration,Latitude,Longitude,Heading,Speed,Distance,Deployment
0,2022-05-23 20:50:16,rhi,[311.],14970.0,4.0,-999.00000,-999.000000,-999.0,-999.0,0.0,1
1,2022-05-23 20:50:25,ppi,[ 1. 37.],14970.0,28.0,-999.00000,-999.000000,-999.0,-999.0,-999.0,1
2,2022-05-23 20:51:01,rhi,[311.],14970.0,4.0,-999.00000,-999.000000,-999.0,-999.0,-999.0,1
3,2022-05-23 20:51:09,ppi,[ 1. 37.],14970.0,28.0,-999.00000,-999.000000,-999.0,-999.0,-999.0,1
4,2022-05-23 20:52:33,rhi,[311.],14970.0,4.0,-999.00000,-999.000000,-999.0,-999.0,-999.0,1
...,...,...,...,...,...,...,...,...,...,...,...
320,2022-05-24 03:12:51,ppi,[0.5 1. ],14970.0,26.0,33.59253,-102.035702,101.0,-999.0,-999.0,8
321,2022-05-24 03:13:18,ppi,[0.5 1. ],14970.0,24.0,33.59253,-102.035702,101.0,-999.0,-999.0,8
322,2022-05-24 03:13:45,ppi,[0.5 1. ],14970.0,22.0,33.59253,-102.035702,101.0,-999.0,-999.0,8
323,2022-05-24 03:14:12,ppi,[0.5 1. ],14970.0,26.0,33.59253,-102.035702,101.0,-999.0,-999.0,8


In [151]:
df2_fixed_indeces = df2[(df2.Latitude > -999) & (df2.Speed > -999) & (df2.Distance > -999)]

In [152]:
df2_fixed_indeces

,Datetime,Scantype,Angles,Ranges,Duration,Latitude,Longitude,Heading,Speed,Distance,Deployment
9,2022-05-23 22:08:09,vad,[37. 45.],9972.0,27.0,33.814864,-102.761356,0.742500,64.902500,997.178167,0
10,2022-05-23 22:08:39,vad,[37. 45.],9972.0,27.0,33.824010,-102.761216,0.731429,65.108214,1018.160963,0
11,2022-05-23 22:09:10,vad,[37. 45.],9972.0,27.0,33.833354,-102.761075,0.696429,65.112500,1040.225420,0
12,2022-05-23 22:09:41,vad,[37. 45.],9972.0,27.0,33.842699,-102.760937,0.692857,65.289643,1040.376307,0
13,2022-05-23 22:10:11,vad,[37. 45.],9972.0,27.0,33.851941,-102.760795,0.712500,67.227500,1028.881233,0
...,...,...,...,...,...,...,...,...,...,...,...
279,2022-05-24 02:52:23,vad,[37. 45.],9972.0,26.0,33.589236,-102.030135,224.631852,25.792963,392.766316,0
280,2022-05-24 02:52:53,vad,[37. 45.],9972.0,27.0,33.587880,-102.032268,281.065357,13.443571,248.799599,0
281,2022-05-24 02:53:23,vad,[37. 45.],9972.0,27.0,33.590467,-102.032914,295.378571,27.913214,294.156356,0
282,2022-05-24 02:53:54,vad,[37. 45.],9972.0,28.0,33.592787,-102.032944,277.806667,12.336667,258.205393,0


In [308]:
scantype_ka2 = df2.Scantype
datetime_ka2 = df2.Datetime
datetime_ka2 = datetime_ka2.astype('datetime64[s]')
angles_ka2 = df2.Angles
ranges_ka2 = df2.Ranges

In [309]:
np.shape(scantype_ka2)

(325,)

In [310]:
datetime_ka2

0     2022-05-23 20:50:16
1     2022-05-23 20:50:25
2     2022-05-23 20:51:01
3     2022-05-23 20:51:09
4     2022-05-23 20:52:33
              ...        
320   2022-05-24 03:12:51
321   2022-05-24 03:13:18
322   2022-05-24 03:13:45
323   2022-05-24 03:14:12
324   2022-05-24 03:14:39
Name: Datetime, Length: 325, dtype: datetime64[s]

In [311]:
scanlog_ka2 = pd.DataFrame(pd.concat([datetime_ka2, scantype_ka2, angles_ka2, ranges_ka2], axis=1))

In [312]:
scanlog_ka2

,Datetime,Scantype,Angles,Ranges
0,2022-05-23 20:50:16,rhi,[311.],14970.0
1,2022-05-23 20:50:25,ppi,[ 1. 37.],14970.0
2,2022-05-23 20:51:01,rhi,[311.],14970.0
3,2022-05-23 20:51:09,ppi,[ 1. 37.],14970.0
4,2022-05-23 20:52:33,rhi,[311.],14970.0
...,...,...,...,...
320,2022-05-24 03:12:51,ppi,[0.5 1. ],14970.0
321,2022-05-24 03:13:18,ppi,[0.5 1. ],14970.0
322,2022-05-24 03:13:45,ppi,[0.5 1. ],14970.0
323,2022-05-24 03:14:12,ppi,[0.5 1. ],14970.0


In [247]:
scanlog_ka2_only_vads = scanlog_ka2[(scanlog_ka2.Scantype == 'vad')]

In [257]:
scanlog_ka2_only_vads_reset_index = scanlog_ka2_only_vads.reset_index()

In [258]:
scanlog_ka2_only_vads_reset_index

,index,Datetime,Scantype,Angles,Ranges
0,9,2022-05-23 22:08:09,vad,[37. 45.],9972.0
1,10,2022-05-23 22:08:39,vad,[37. 45.],9972.0
2,11,2022-05-23 22:09:10,vad,[37. 45.],9972.0
3,12,2022-05-23 22:09:41,vad,[37. 45.],9972.0
4,13,2022-05-23 22:10:11,vad,[37. 45.],9972.0
...,...,...,...,...,...
161,279,2022-05-24 02:52:23,vad,[37. 45.],9972.0
162,280,2022-05-24 02:52:53,vad,[37. 45.],9972.0
163,281,2022-05-24 02:53:23,vad,[37. 45.],9972.0
164,282,2022-05-24 02:53:54,vad,[37. 45.],9972.0


In [313]:
radar_column_ka2 = pd.Series(radar_array_ka2)
velmean_column_ka2 = pd.Series(velmean_array_ka2)
velvar_column_ka2 = pd.Series(velvar_array_ka2)
bearmean_column_ka2 = pd.Series(bearmean_array_ka2)
latmean_column_ka2 = pd.Series(latmean_array_ka2)
latvar_column_ka2 = pd.Series(latvar_array_ka2)
lonmean_column_ka2 = pd.Series(lonmean_array_ka2)
lonvar_column_ka2 = pd.Series(lonvar_array_ka2)
datetime_column_ka2 = pd.Series(array_dealiased_vad_ka2_times_datetime)

In [314]:
datetime_column_ka2

0     2022-05-23 22:07:37
1     2022-05-23 22:08:09
2     2022-05-23 22:08:39
3     2022-05-23 22:09:10
4     2022-05-23 22:09:41
              ...        
253   2022-05-24 02:51:52
254   2022-05-24 02:52:23
255   2022-05-24 02:52:53
256   2022-05-24 02:53:23
257   2022-05-24 02:53:54
Length: 258, dtype: datetime64[s]

In [272]:
for 
corrected_scans_df_ka2 = pd.DataFrame(pd.concat([scanlog_ka2_only_vads_reset_index.Datetime, scanlog_ka2_only_vads_reset_index.Scantype, 
                                                 scanlog_ka2_only_vads_reset_index.Angles, scanlog_ka2_only_vads_reset_index.Ranges,
                                                 datetime_column_ka2, radar_column_ka2, velmean_column_ka2, 
                                                 velvar_column_ka2, bearmean_column_ka2, latmean_column_ka2, latvar_column_ka2,
                                                 lonmean_column_ka2, lonvar_column_ka2], axis=1))

In [273]:
corrected_scans_df_ka2

,Datetime,Scantype,Angles,Ranges,0,1,2,3,4,5,6,7,8
0,2022-05-23 22:08:09,vad,[37. 45.],9972.0,2022-05-23 22:08:09,<pyart.core.radar.Radar object at 0x30005e370>,65.260213,1.055541,0.732340,33.817767,1.747907e-05,-102.761312,4.154278e-09
1,2022-05-23 22:08:39,vad,[37. 45.],9972.0,2022-05-23 22:08:39,<pyart.core.radar.Radar object at 0x105b98460>,65.172083,0.489455,0.720625,33.827022,1.782513e-05,-102.761170,4.179998e-09
2,2022-05-23 22:09:10,vad,[37. 45.],9972.0,2022-05-23 22:09:10,<pyart.core.radar.Radar object at 0x306a46490>,65.030000,0.118657,0.705957,33.836214,1.706878e-05,-102.761033,3.670925e-09
3,2022-05-23 22:09:41,vad,[37. 45.],9972.0,2022-05-23 22:09:41,<pyart.core.radar.Radar object at 0x306b718b0>,66.019149,0.997325,0.727447,33.845616,1.765445e-05,-102.760892,4.144279e-09
4,2022-05-23 22:10:11,vad,[37. 45.],9972.0,2022-05-23 22:10:11,<pyart.core.radar.Radar object at 0x306e20580>,67.203191,0.195279,0.728511,33.854908,1.833633e-05,-102.760750,4.307415e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,NaT,NaN,NaN,NaN,2022-05-24 02:51:52,<pyart.core.radar.Radar object at 0x331cbc2b0>,24.496667,12.280095,207.475778,33.591246,1.474247e-06,-102.028183,8.916879e-07
253,NaT,NaN,NaN,NaN,2022-05-24 02:52:23,<pyart.core.radar.Radar object at 0x331cb4370>,20.255778,77.901670,231.209556,33.588663,8.279943e-07,-102.030829,1.195708e-06
254,NaT,NaN,NaN,NaN,2022-05-24 02:52:53,<pyart.core.radar.Radar object at 0x331cc1b20>,18.721739,62.057095,280.521957,33.588523,8.732854e-07,-102.032520,1.990472e-07
255,NaT,NaN,NaN,NaN,2022-05-24 02:53:23,<pyart.core.radar.Radar object at 0x31fedf700>,24.807838,40.677301,278.348378,33.591005,1.869667e-06,-102.032919,3.519970e-10


In [274]:
corrected_scans_df_ka2.rename(columns={0: 'Datetime_VAD', 1: 'Radar', 2: 'Velmean', 3: 'Velvar', 4: 'Bearmean', 
                                        5: 'Latmean', 6: 'Latvar', 7: 'Lonmean', 8: 'Lonvar'}, inplace  = True)

In [275]:
corrected_scans_df_ka2 = corrected_scans_df_ka2.dropna()

In [276]:
corrected_scans_df_ka2

,Datetime,Scantype,Angles,Ranges,Datetime_VAD,Radar,Velmean,Velvar,Bearmean,Latmean,Latvar,Lonmean,Lonvar
0,2022-05-23 22:08:09,vad,[37. 45.],9972.0,2022-05-23 22:08:09,<pyart.core.radar.Radar object at 0x30005e370>,65.260213,1.055541,0.732340,33.817767,1.747907e-05,-102.761312,4.154278e-09
1,2022-05-23 22:08:39,vad,[37. 45.],9972.0,2022-05-23 22:08:39,<pyart.core.radar.Radar object at 0x105b98460>,65.172083,0.489455,0.720625,33.827022,1.782513e-05,-102.761170,4.179998e-09
2,2022-05-23 22:09:10,vad,[37. 45.],9972.0,2022-05-23 22:09:10,<pyart.core.radar.Radar object at 0x306a46490>,65.030000,0.118657,0.705957,33.836214,1.706878e-05,-102.761033,3.670925e-09
3,2022-05-23 22:09:41,vad,[37. 45.],9972.0,2022-05-23 22:09:41,<pyart.core.radar.Radar object at 0x306b718b0>,66.019149,0.997325,0.727447,33.845616,1.765445e-05,-102.760892,4.144279e-09
4,2022-05-23 22:10:11,vad,[37. 45.],9972.0,2022-05-23 22:10:11,<pyart.core.radar.Radar object at 0x306e20580>,67.203191,0.195279,0.728511,33.854908,1.833633e-05,-102.760750,4.307415e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,2022-05-24 02:52:23,vad,[37. 45.],9972.0,2022-05-24 01:23:15,<pyart.core.radar.Radar object at 0x31a7e1820>,0.000000,0.000000,118.880000,33.706523,5.166122e-29,-102.530607,8.265795e-28
162,2022-05-24 02:52:53,vad,[37. 45.],9972.0,2022-05-24 01:24:52,<pyart.core.radar.Radar object at 0x31a7b1610>,0.001133,0.000096,118.880000,33.706528,4.475392e-12,-102.530610,2.170022e-12
163,2022-05-24 02:53:23,vad,[37. 45.],9972.0,2022-05-24 01:27:04,<pyart.core.radar.Radar object at 0x31a4213a0>,0.000000,0.000000,118.880000,33.706531,9.754813e-13,-102.530612,2.776089e-14
164,2022-05-24 02:53:54,vad,[37. 45.],9972.0,2022-05-24 01:29:15,<pyart.core.radar.Radar object at 0x31a4218e0>,0.000000,0.000000,118.880000,33.706533,2.518236e-13,-102.530613,4.309281e-14


In [282]:
corrected_vads_only_df_ka2 = corrected_scans_df_ka2

In [34]:
# path = '/Users/juliabman/Desktop/vads_radar_objects/ka1/05232022'
# for vad in corrected_vads_only_df_ka1.Radar:
#     print(vad)
#     vad_filename_split = vad.time['units'][14:24].split('-') + vad.time['units'][25:-1].split(':')
#     vad_filename = ''.join(vad_filename_split)
#     print(vad_filename)
#     pyart.io.write_cfradial(f"/Users/juliabman/Desktop/not_terminal_corrected/ka1/05232022/{vad_filename}.nc", vad)

In [35]:
# path = '/Users/juliabman/Desktop/vads_radar_objects/ka2/05232022'
# for vad in corrected_vads_only_df_ka2.Radar:
#     print(vad)
#     vad_filename_split = vad.time['units'][14:24].split('-') + vad.time['units'][25:-1].split(':')
#     vad_filename = ''.join(vad_filename_split)
#     print(vad_filename)
#     pyart.io.write_cfradial(f"/Users/juliabman/Desktop/not_terminal_corrected/ka2/05232022/{vad_filename}.nc", vad)

In [36]:
def find_nearest(array, value):

    array = np.asarray(array)
    idx = (np.abs(array-value)).argmin()
    return idx

In [37]:
tobac_times_closest_to_ka1_indeces = []
time_array = np.array(corrected_vads_only_df_ka1.Datetime)

for a_ka_time in time_array:
    #time = pd.Timestamp.to_pydatetime(corrected_vads_only_df_ka1.Datetime[a_ka_time])
    #datetime_proper = datetime.strftime(time)
    #print(a_ka_time)
    nearest_tobac_index = find_nearest(morton_tobac_times_datetime.data, a_ka_time)
    tobac_times_closest_to_ka1_indeces.append(nearest_tobac_index)

In [38]:
tobac_times_closest_ka1 = morton_tobac_times_datetime[tobac_times_closest_to_ka1_indeces]
series_tobac_times = pd.Series(tobac_times_closest_ka1)
series_tobac_indeces = pd.Series(tobac_times_closest_to_ka1_indeces)

tobac_lats_closest_ka1 = morton_tobac_lats[tobac_times_closest_to_ka1_indeces]
series_tobac_lats_ka1 = pd.Series(tobac_lats_closest_ka1)
tobac_lons_closest_ka1 = morton_tobac_lons[tobac_times_closest_to_ka1_indeces]
series_tobac_lons_ka1 = pd.Series(tobac_lons_closest_ka1)

In [39]:
series_tobac_times

0     2022-05-23 22:21:17
1     2022-05-23 22:21:17
2     2022-05-23 22:21:17
3     2022-05-23 22:21:17
4     2022-05-23 22:21:17
              ...        
150   2022-05-24 00:03:14
151   2022-05-24 00:03:14
152   2022-05-24 00:03:14
153   2022-05-24 00:03:14
154   2022-05-24 00:03:14
Length: 155, dtype: datetime64[ns]

In [40]:
reset_indeces_vads_ka1 = corrected_vads_only_df_ka1.reset_index()

In [41]:
corrected_vads_only_df_ka1_plus_tobac = pd.DataFrame(pd.concat([reset_indeces_vads_ka1, series_tobac_times, series_tobac_indeces, 
                                                                series_tobac_lats_ka1, series_tobac_lons_ka1],axis=1))

In [42]:
corrected_vads_only_df_ka1_plus_tobac.rename(columns={0: 'tobac_time', 1: 'tobac_time_indeces', 2: 'tobac_lats', 3: 'tobac_lons'}, inplace = True)

In [43]:
ka1_vads_df = corrected_vads_only_df_ka1_plus_tobac

In [283]:
tobac_times_closest_to_ka2_indeces = []
time_array_2 = np.array(corrected_vads_only_df_ka2.Datetime)

for a_ka_time_2 in time_array_2:
    #time = pd.Timestamp.to_pydatetime(corrected_vads_only_df_ka1.Datetime[a_ka_time])
    #datetime_proper = datetime.strftime(time)
    #print(a_ka_time)
    nearest_tobac_index_2 = find_nearest(morton_tobac_times_datetime.data, a_ka_time_2)
    tobac_times_closest_to_ka2_indeces.append(nearest_tobac_index_2)

In [284]:
tobac_times_closest_ka2 = morton_tobac_times_datetime[tobac_times_closest_to_ka2_indeces]
series_tobac_times_2 = pd.Series(tobac_times_closest_ka2)
series_tobac_indeces_2 = pd.Series(tobac_times_closest_to_ka2_indeces)

tobac_lats_closest_ka2 = morton_tobac_lats[tobac_times_closest_to_ka2_indeces]
series_tobac_lats_ka2 = pd.Series(tobac_lats_closest_ka2)
tobac_lons_closest_ka2 = morton_tobac_lons[tobac_times_closest_to_ka2_indeces]
series_tobac_lons_ka2 = pd.Series(tobac_lons_closest_ka2)

In [285]:
reset_indeces_vads_ka2 = corrected_vads_only_df_ka2.reset_index()
corrected_vads_only_df_ka2_plus_tobac = pd.DataFrame(pd.concat([reset_indeces_vads_ka2, series_tobac_times_2, series_tobac_indeces_2, 
                                                                series_tobac_lats_ka2, series_tobac_lons_ka2],axis=1))
corrected_vads_only_df_ka2_plus_tobac.rename(columns={0: 'tobac_time', 1: 'tobac_time_indeces', 2: 'tobac_lats', 3: 'tobac_lons'}, inplace = True)

In [297]:
ka2_vads_df = corrected_vads_only_df_ka2_plus_tobac

In [298]:
ka2_vads_df

,index,Datetime,Scantype,Angles,Ranges,Datetime_VAD,Radar,Velmean,Velvar,Bearmean,Latmean,Latvar,Lonmean,Lonvar,tobac_time,tobac_time_indeces,tobac_lats,tobac_lons,Distance_From_Storm
0,0,2022-05-23 22:08:09,vad,[37. 45.],9972.0,2022-05-23 22:08:09,<pyart.core.radar.Radar object at 0x30005e370>,65.260213,1.055541,0.732340,33.817767,1.747907e-05,-102.761312,4.154278e-09,2022-05-23 22:21:17,0,33.725464,-103.115808,34.375022
1,1,2022-05-23 22:08:39,vad,[37. 45.],9972.0,2022-05-23 22:08:39,<pyart.core.radar.Radar object at 0x105b98460>,65.172083,0.489455,0.720625,33.827022,1.782513e-05,-102.761170,4.179998e-09,2022-05-23 22:21:17,0,33.725464,-103.115808,34.707656
2,2,2022-05-23 22:09:10,vad,[37. 45.],9972.0,2022-05-23 22:09:10,<pyart.core.radar.Radar object at 0x306a46490>,65.030000,0.118657,0.705957,33.836214,1.706878e-05,-102.761033,3.670925e-09,2022-05-23 22:21:17,0,33.725464,-103.115808,35.064583
3,3,2022-05-23 22:09:41,vad,[37. 45.],9972.0,2022-05-23 22:09:41,<pyart.core.radar.Radar object at 0x306b718b0>,66.019149,0.997325,0.727447,33.845616,1.765445e-05,-102.760892,4.144279e-09,2022-05-23 22:21:17,0,33.725464,-103.115808,35.456437
4,4,2022-05-23 22:10:11,vad,[37. 45.],9972.0,2022-05-23 22:10:11,<pyart.core.radar.Radar object at 0x306e20580>,67.203191,0.195279,0.728511,33.854908,1.833633e-05,-102.760750,4.307415e-09,2022-05-23 22:21:17,0,33.725464,-103.115808,35.869927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,161,2022-05-24 02:52:23,vad,[37. 45.],9972.0,2022-05-24 01:23:15,<pyart.core.radar.Radar object at 0x31a7e1820>,0.000000,0.000000,118.880000,33.706523,5.166122e-29,-102.530607,8.265795e-28,2022-05-24 02:54:14,45,33.686458,-102.129656,37.201745
162,162,2022-05-24 02:52:53,vad,[37. 45.],9972.0,2022-05-24 01:24:52,<pyart.core.radar.Radar object at 0x31a7b1610>,0.001133,0.000096,118.880000,33.706528,4.475392e-12,-102.530610,2.170022e-12,2022-05-24 02:54:14,45,33.686458,-102.129656,37.202062
163,163,2022-05-24 02:53:23,vad,[37. 45.],9972.0,2022-05-24 01:27:04,<pyart.core.radar.Radar object at 0x31a4213a0>,0.000000,0.000000,118.880000,33.706531,9.754813e-13,-102.530612,2.776089e-14,2022-05-24 02:54:14,45,33.686458,-102.129656,37.202261
164,164,2022-05-24 02:53:54,vad,[37. 45.],9972.0,2022-05-24 01:29:15,<pyart.core.radar.Radar object at 0x31a4218e0>,0.000000,0.000000,118.880000,33.706533,2.518236e-13,-102.530613,4.309281e-14,2022-05-24 02:54:14,45,33.686458,-102.129656,37.202357


In [71]:
# from stackoverflow (https://stackoverflow.com/questions/639695/how-to-convert-latitude-or-longitude-to-meters)
# originally in javascript
def measure(lat1, lon1, lat2, lon2):  # generally used geo measurement function
    R = 6378.137 # Radius of earth in KM
    dLat = lat2 * np.pi / 180 - lat1 * np.pi / 180;
    dLon = lon2 * np.pi / 180 - lon1 * np.pi / 180;
    a = np.sin(dLat/2) * np.sin(dLat/2) + np.cos(lat1 * np.pi / 180) * np.cos(lat2 * np.pi / 180) * np.sin(dLon/2) * np.sin(dLon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c
    #d_meters = d * 1000
    return d # km

In [288]:
distance_ka1_from_storm = measure(corrected_vads_only_df_ka1_plus_tobac.tobac_lats, corrected_vads_only_df_ka1_plus_tobac.tobac_lons, 
                                  corrected_vads_only_df_ka1_plus_tobac.Latmean, corrected_vads_only_df_ka1_plus_tobac.Lonmean)

In [299]:
distance_ka2_from_storm = measure(ka2_vads_df.tobac_lats, ka2_vads_df.tobac_lons, 
                                  ka2_vads_df.Latmean, ka2_vads_df.Lonmean)

In [300]:
distance_ka1_from_storm_series = pd.Series(distance_ka1_from_storm)
distance_ka2_from_storm_series = pd.Series(distance_ka2_from_storm)

In [301]:
distance_ka2_from_storm_series

0      34.375022
1      34.707656
2      35.064583
3      35.456437
4      35.869927
         ...    
161    37.201745
162    37.202062
163    37.202261
164    37.202357
165    29.845491
Length: 166, dtype: float64

In [76]:
ka1_vads_df = ka1_vads_df.insert(18, 'Distance_From_Storm', distance_ka1_from_storm_series)

ValueError: cannot insert Distance_From_Storm, already exists

In [294]:
#ka2_vads_df.drop(columns = 'Distance_From_Storm')

In [302]:
ka2_vads_df = ka2_vads_df.insert(18, 'Distance_From_Storm', distance_ka2_from_storm_series)

ValueError: cannot insert Distance_From_Storm, already exists

In [78]:
ka1_vads_df

,index,Datetime,Datetime_VAD,Scantype,Angles,Ranges,Radar,Velmean,Velvar,Bearmean,Latmean,Latvar,Lonmean,Lonvar,tobac_time,tobac_time_indeces,tobac_lats,tobac_lons,Distance_From_Storm
0,0,2022-05-23 20:18:19,2022-05-23 20:18:19,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30000b6a0>,41.322128,13.541626,270.857660,33.593844,1.493618e-09,-102.341872,9.543316e-06,2022-05-23 22:21:17,0,33.725464,-103.115808,73.191288
1,1,2022-05-23 20:18:49,2022-05-23 20:18:49,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30013a8b0>,42.164255,1.027729,270.870426,33.593926,1.237346e-09,-102.348762,1.018093e-05,2022-05-23 22:21:17,0,33.725464,-103.115808,72.564113
2,2,2022-05-23 20:19:19,2022-05-23 20:19:19,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30005eb80>,39.817556,15.065437,270.855778,33.594000,1.142237e-09,-102.355451,8.773930e-06,2022-05-23 22:21:17,0,33.725464,-103.115808,71.955495
3,3,2022-05-23 20:19:48,2022-05-23 20:19:48,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x105b14880>,2.022109,46.544900,298.949818,33.594931,5.005760e-08,-102.361993,4.267158e-07,2022-05-23 22:21:17,0,33.725464,-103.115808,71.340527
4,4,2022-05-23 20:20:19,2022-05-23 20:20:19,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x30073d070>,6.971087,18.566854,232.127826,33.594825,3.518888e-08,-102.361671,1.986989e-07,2022-05-23 22:21:17,0,33.725464,-103.115808,71.372215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,190,2022-05-24 00:01:10,2022-05-24 00:01:10,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x307afa580>,3.976230,137.229424,248.844976,33.793174,1.025092e-05,-102.644966,1.008538e-06,2022-05-24 00:03:14,18,33.900867,-102.732445,14.461411
151,191,2022-05-24 00:01:39,2022-05-24 00:01:39,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x307d398b0>,43.196667,15.037367,38.183750,33.784412,8.276087e-06,-102.642297,2.191911e-09,2022-05-24 00:03:14,18,33.900867,-102.732445,15.412055
152,192,2022-05-24 00:02:09,2022-05-24 00:02:09,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x307c26ac0>,30.318413,305.878072,94.438095,33.791226,7.861599e-06,-102.642176,2.957845e-09,2022-05-24 00:03:14,18,33.900867,-102.732445,14.785755
153,193,2022-05-24 00:02:39,2022-05-24 00:02:39,vad,[37. 45.],9972.0,<pyart.core.radar.Radar object at 0x307c267f0>,15.259574,295.846113,167.150213,33.793709,3.987920e-07,-102.642206,1.600216e-08,2022-05-24 00:03:14,18,33.900867,-102.732445,14.556815


In [303]:
ka2_vads_df

,index,Datetime,Scantype,Angles,Ranges,Datetime_VAD,Radar,Velmean,Velvar,Bearmean,Latmean,Latvar,Lonmean,Lonvar,tobac_time,tobac_time_indeces,tobac_lats,tobac_lons,Distance_From_Storm
0,0,2022-05-23 22:08:09,vad,[37. 45.],9972.0,2022-05-23 22:08:09,<pyart.core.radar.Radar object at 0x30005e370>,65.260213,1.055541,0.732340,33.817767,1.747907e-05,-102.761312,4.154278e-09,2022-05-23 22:21:17,0,33.725464,-103.115808,34.375022
1,1,2022-05-23 22:08:39,vad,[37. 45.],9972.0,2022-05-23 22:08:39,<pyart.core.radar.Radar object at 0x105b98460>,65.172083,0.489455,0.720625,33.827022,1.782513e-05,-102.761170,4.179998e-09,2022-05-23 22:21:17,0,33.725464,-103.115808,34.707656
2,2,2022-05-23 22:09:10,vad,[37. 45.],9972.0,2022-05-23 22:09:10,<pyart.core.radar.Radar object at 0x306a46490>,65.030000,0.118657,0.705957,33.836214,1.706878e-05,-102.761033,3.670925e-09,2022-05-23 22:21:17,0,33.725464,-103.115808,35.064583
3,3,2022-05-23 22:09:41,vad,[37. 45.],9972.0,2022-05-23 22:09:41,<pyart.core.radar.Radar object at 0x306b718b0>,66.019149,0.997325,0.727447,33.845616,1.765445e-05,-102.760892,4.144279e-09,2022-05-23 22:21:17,0,33.725464,-103.115808,35.456437
4,4,2022-05-23 22:10:11,vad,[37. 45.],9972.0,2022-05-23 22:10:11,<pyart.core.radar.Radar object at 0x306e20580>,67.203191,0.195279,0.728511,33.854908,1.833633e-05,-102.760750,4.307415e-09,2022-05-23 22:21:17,0,33.725464,-103.115808,35.869927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,161,2022-05-24 02:52:23,vad,[37. 45.],9972.0,2022-05-24 01:23:15,<pyart.core.radar.Radar object at 0x31a7e1820>,0.000000,0.000000,118.880000,33.706523,5.166122e-29,-102.530607,8.265795e-28,2022-05-24 02:54:14,45,33.686458,-102.129656,37.201745
162,162,2022-05-24 02:52:53,vad,[37. 45.],9972.0,2022-05-24 01:24:52,<pyart.core.radar.Radar object at 0x31a7b1610>,0.001133,0.000096,118.880000,33.706528,4.475392e-12,-102.530610,2.170022e-12,2022-05-24 02:54:14,45,33.686458,-102.129656,37.202062
163,163,2022-05-24 02:53:23,vad,[37. 45.],9972.0,2022-05-24 01:27:04,<pyart.core.radar.Radar object at 0x31a4213a0>,0.000000,0.000000,118.880000,33.706531,9.754813e-13,-102.530612,2.776089e-14,2022-05-24 02:54:14,45,33.686458,-102.129656,37.202261
164,164,2022-05-24 02:53:54,vad,[37. 45.],9972.0,2022-05-24 01:29:15,<pyart.core.radar.Radar object at 0x31a4218e0>,0.000000,0.000000,118.880000,33.706533,2.518236e-13,-102.530613,4.309281e-14,2022-05-24 02:54:14,45,33.686458,-102.129656,37.202357


In [80]:
ka1_vads_df.to_csv(path_or_buf = '/Users/juliabman/Desktop/research2024/vad_dfs/ka1_vads_df')

In [81]:
ka2_vads_df.to_csv(path_or_buf = '/Users/juliabman/Desktop/research2024/vad_dfs/ka2_vads_df')